In [1]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env();

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [3]:
G_max = 74.0 #max gradient, prevent nerve stimulation by limiting max gradient and slew rate. mT/m
SR_max = 50.0 #max slew rate mT/m/ms

dt = 0.5 #ms
Teps = 26.4 #time to echo ms
T90 = 5.3 #90 degree RF time ms
T180 = 4.3 #half of 180 degree RF time ms
lambda_null = 60 #ms
epsilion = 1e-4
GAMMA = (42.58*10^3) * 2 * pi;


TEmin = T180 + 2 * Teps
TEmax = 97.0
bval = 500.0;

In [4]:
Teps = trunc(Int,Teps/dt+0.5)
T90 = trunc(Int, T90/dt + 0.5)
T180 = trunc(Int, T180/dt/2 + 0.5)
TEmin = trunc(Int, TEmin/dt + 0.5)
TEmax = trunc(Int, TEmax/dt + 0.5)
lambda_null = lambda_null/dt;

In [5]:
#variable
TE = TEmax + 1
iter = 0

solve_time_all = []
obj_all = []
Gp_all = []
Gn_all = []
F_all = []

while true
    TE -= 1
    iter += 1
    Tdiff = TE-Teps
    TEH = trunc(Int, TE/2.0 + 0.5)
    
    Inv = -1.0* ones(TE)
    Inv[1:TEH] .= 1.0
    
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(model, "MIPGap" => 1e-4,"OutputFlag" => 1);

    @variable(model, Gp[1:TE]>=0) #Gradient positive part before 180 pulse
    @variable(model, Gn[1:TE]>=0) #Gradient positive part after 180 pulse
    @variable(model, G_sign[1:TE], Bin) #Gradient sign (only positive or negative)
    @variable(model, F[1:TE]) #F value
    @variable(model, F_sign[1:TE], Bin) #F value
    @variable(model, Fp[1:TE]>=0) #F value
    @variable(model, Fn[1:TE]>=0) #F value
    
    @objective(model, Max, sum(Fp[i]+Fn[i] for i=1:TE))
    
    @constraint(model, Gp_max_const[i in 1:TE], Gp[i] <= G_max*G_sign[i])
    @constraint(model, Gn_max_const[i in 1:TE], Gn[i] <= G_max*(1-G_sign[i]))
    @constraint(model, max_slew_const1[i in 2:TE], -SR_max*dt<= Gp[i]-Gp[i-1] <= SR_max*dt)
    @constraint(model, max_slew_const2[i in 2:TE], -SR_max*dt<= Gn[i]-Gn[i-1] <= SR_max*dt)
    @constraint(model, T90_const1[i in 1:T90], Gp[i] == 0.0)
    @constraint(model, T90_const2[i in 1:T90], Gn[i] == 0.0)
    
    @constraint(model, T180_const1[i in TEH-T180+1:TEH+T180], Gp[i] == 0.0)
    @constraint(model, TEPI_const1[i in Tdiff+1:TE], Gp[i] == 0.0)
    @constraint(model, T180_const2[i in TEH-T180+1:TEH+T180], Gn[i] == 0.0)
    @constraint(model, TEPI_const2[i in Tdiff+1:TE], Gn[i] == 0.0)

    @constraint(model, M0_const, sum(Inv[i]*(Gp[i]-Gn[i]) for i = 1:TE) == 0.0)
    @constraint(model, M1_const, sum(i*Inv[i]*(Gp[i]-Gn[i]) for i = 1:TE) == 0.0)
    
    @constraint(model, F_const1, F[1] == 0)
    @constraint(model, F_const2[i in 2:TE], F[i] == sum(Inv[j] * (Gp[j] - Gn[j]) for j=1:i)*0.001*0.001 *dt)
    @constraint(model, F_const_abs1[i in 1:TE], Fp[i]-Fn[i] == F[i])
    @constraint(model, F_const_abs2[i in 1:TE], Fp[i] <= 10 * F_sign[i])
    @constraint(model, F_const_abs3[i in 1:TE], Fn[i] <= 10 * (1 - F_sign[i]))

    push!(solve_time_all, @elapsed optimize!(model))
    push!(obj_all, objective_value(model))
    
    b = 0
    
    for i in 1:TE
        b = b + value.(F)[i]^2
    end
    
    b = b * GAMMA * GAMMA *0.001*dt
    
    @printf("TEH: %.2f, Tdiff: %.2f, TE: %.2f, bval: %.2f\n", TEH*dt, Tdiff*dt, TE*dt, b)

    push!(Gp_all, value.(Gp))
    push!(Gn_all, value.(Gn))
    push!(F_all, value.(F))

    if(b >= bval)
        break
    end
    
    
end
    

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1696 rows, 1744 columns and 42234 nonzeros
Model fingerprint: 0x08996cd0
Variable types: 1356 continuous, 388 integer (388 binary)
Coefficient statistics:
  Matrix range     [5e-07, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [1e+01, 7e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 648 rows and 584 columns
Presolve time: 0.01s
Presolved: 1048 rows, 1160 columns, 4319 nonzeros
Variable types: 855 continuous, 305 integer (305 binary)

Root relaxation: objective 5.253570e-01, 570 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.52536    0  179   -0.00000    0.52536      -     -    0s
H    0     0   

In [6]:
TEmin*dt

57.0

In [7]:
plot(F_all[47])

LoadError: BoundsError: attempt to access 1-element Vector{Any} at index [47]

In [8]:
T180

4

In [9]:
sum(solve_time_all)

55.203346

In [10]:
TE*dt

97.0

In [11]:
Gp_all

1-element Vector{Any}:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [12]:
G_ap2

LoadError: UndefVarError: G_ap2 not defined

In [13]:
TE = TEmax + 1
iter = 0

solve_time_all2 = []
obj_all2 = []
Gp_all2 = []
Gn_all2 = []
F_all2 = []

while true
    TE -= 1
    iter += 1
    Tdiff = TE-Teps
    TEH = trunc(Int, TE/2.0 + 0.5)
    
    Inv = -1.0* ones(TE)
    Inv[1:TEH] .= 1.0
    
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(model, "MIPGap" => 1e-4,"OutputFlag" => 1);

    @variable(model, Gp[1:TE]>=0) #Gradient positive part before 180 pulse
    @variable(model, Gn[1:TE]>=0) #Gradient positive part after 180 pulse
    @variable(model, G_sign[1:TE], Bin) #Gradient sign (only positive or negative)
    @variable(model, F[1:TE]) #F value
    
    @objective(model, Max, sum(Gp[i]+Gn[i] for i=1:TE))
    
    @constraint(model, Gp_max_const[i in 1:TE], Gp[i] <= G_max*G_sign[i])
    @constraint(model, Gn_max_const[i in 1:TE], Gn[i] <= G_max*(1-G_sign[i]))
    @constraint(model, max_slew_const1[i in 2:TE], -SR_max*dt<= Gp[i]-Gp[i-1] <= SR_max*dt)
    @constraint(model, max_slew_const2[i in 2:TE], -SR_max*dt<= Gn[i]-Gn[i-1] <= SR_max*dt)
    @constraint(model, T90_const1[i in 1:T90], Gp[i] == 0.0)
    @constraint(model, T90_const2[i in 1:T90], Gn[i] == 0.0)
    
    @constraint(model, T180_const1[i in TEH-T180+1:TEH+T180], Gp[i] == 0.0)
    @constraint(model, TEPI_const1[i in Tdiff+1:TE], Gp[i] == 0.0)
    @constraint(model, T180_const2[i in TEH-T180+1:TEH+T180], Gn[i] == 0.0)
    @constraint(model, TEPI_const2[i in Tdiff+1:TE], Gn[i] == 0.0)

    @constraint(model, M0_const, sum(Inv[i]*(Gp[i]-Gn[i]) for i = 1:TE) == 0.0)
    @constraint(model, M1_const, sum(i*Inv[i]*(Gp[i]-Gn[i]) for i = 1:TE) == 0.0)
    
    @constraint(model, F_const1, F[1] == 0)
    @constraint(model, F_const2[i in 2:TE], F[i] == sum(Inv[j] * (Gp[j] - Gn[j]) for j=1:i)*0.001*0.001 *dt)

    push!(solve_time_all2, @elapsed optimize!(model))
    push!(obj_all2, objective_value(model))
    
    b = 0
    
    for i in 1:TE
        b = b + value.(F)[i]^2
    end
    
    b = b * GAMMA * GAMMA *0.001*dt
    
    @printf("TEH: %.2f, Tdiff: %.2f, TE: %.2f, bval: %.2f\n", TEH*dt, Tdiff*dt, TE*dt, b)

    push!(Gp_all2, value.(Gp))
    push!(Gn_all2, value.(Gn))
    push!(F_all2, value.(F))

    if(b >= bval)
        break
    end
    
    
end

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1114 rows, 1162 columns and 40876 nonzeros
Model fingerprint: 0x4f39c1a2
Variable types: 968 continuous, 194 integer (194 binary)
Coefficient statistics:
  Matrix range     [5e-07, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+01, 2e+01]
  RHS range        [7e+01, 7e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 628 rows and 564 columns
Presolve time: 0.00s
Presolved: 486 rows, 598 columns, 1694 nonzeros
Variable types: 476 continuous, 122 integer (122 binary)

Root relaxation: objective 8.736000e+03, 320 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8736.00000    0   18   -0.00000 8736.00000      -     -    0s
H    0     0      

In [14]:
G_ap1 = Gp_all[47]-Gn_all[47]
G_ap2 = zeros(74)
for i in 1:74
    G_ap2[i] = G_ap1[1+2*(i-1)]
end

G_ap3 = Gp_all2[47]-Gn_all2[47]
G_ap4 = zeros(74)
for i in 1:74
    G_ap4[i] = G_ap3[1+2*(i-1)]
end

xx = range(1,74)

plot(xx,G_ap2,title="Optimal G(t)-Implementation 2",xtickfontsize=12,ytickfontsize=12,label="bval1",xlabel="Time[ms]",ylabel="Gradient amplitude[mT/m]",fmt = :png)
plot!(xx,G_ap4, label="bval2")

LoadError: BoundsError: attempt to access 1-element Vector{Any} at index [47]

In [15]:
sum(solve_time_all)

55.203346

In [16]:
sum(solve_time_all2)

0.993192542

In [17]:
1080/60

18.0

In [18]:
173/60

2.8833333333333333